All the data transformations that need to happen, from raw to final:
* NHPD Raw -> Change any null values for fields to "Not Available"
* NHPD Carto -> Save as a CSV to reduce storage on Carto server
* NHPD for Report -> Clean the lat lon and save as spatial file 
* DACs -> Join DAC Percentiles with Mapping Display Data. Make sure data types are correct (floats, not strings). Join QCT Data.
  * Carto_DACs -> have as few fields as possible, short names
  * Report_DACs -> have all the fields, named well
* Tribes and Territories -> Get from GIS SQLite. Naming format should be same as Counties and States.

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import sqlite3 as sql
import subprocess
import sys
import os
from os import mkdir
from os.path import exists, sep

In [3]:
# Enter directory paths here
WD = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data"
RAW = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/raw"
SQL_OUTPUT = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/sql_output"
CARTO = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/carto"
REPORT = "/Users/anushreechaudhuri/pCloud Drive/MIT/MIT Work/DC DOE/app_files/equity-tool/data/report"

In [185]:
os.chdir(RAW)
nhpd = pd.read_csv("nhpd.csv")

In [190]:
nhpd = nhpd[['Property Name','Street Address', 'City', 'State', 'Zip Code','Subsidy Status', 'Subsidy Name', 'Subsidy Subname',
       'Start Date', 'End Date', 'Assisted Units', 
       'Owner Name', 'Owner Type', 'Manager Name', 'Manager Type',
       '0-1 Bedroom Units', 'Two Bedroom Units', 'Three+ Bedroom Units',
       'Target Population', 'Earliest Construction Date',
       'Latest Construction Date', 'Rent to FMR Ratio', 'Known Total Units',
       'Renewal Type Name', 'Inactive Status Description', 'Construction Type',
    'Renewal Name', 'Renewal ID', 'SOA Code', 'Latitude', 'Longitude']]
carto_nhpd = nhpd[['Property Name','Street Address', 'City', 'State', 'Zip Code','Subsidy Status', 'Subsidy Name', 'Subsidy Subname',
       'Start Date', 'End Date', 'Assisted Units', 
       'Owner Name', 'Owner Type', 'Manager Name', 'Manager Type',
       '0-1 Bedroom Units', 'Two Bedroom Units', 'Three+ Bedroom Units',
       'Target Population', 'Earliest Construction Date',
       'Latest Construction Date', 'Rent to FMR Ratio', 'Known Total Units',
       'Renewal Type Name', 'Inactive Status Description', 'Construction Type',
    'Renewal Name', 'Renewal ID', 'SOA Code', 'Latitude', 'Longitude']]

In [191]:
# Rename columns in NHPD
carto_nhpd.rename(columns={"Property Name": "name", 'Street Address': 'add', 'City': 'city', 'State': 'state', 'Zip Code': 'zip', 'Subsidy Status': 'status', 'Subsidy Name':'subsidy','Subsidy Subname':'subnm', 'Start Date': 'start', 'End Date': 'end', 'Assisted Units': 'units', 'Owner Name': 'owner', 'Owner Type': 'otype', 'Manager Name': 'mgr', 'Manager Type': 'mtype', '0-1 Bedroom Units': '0-1', 'Two Bedroom Units': '2-4', 'Three+ Bedroom Units': '5-+', 'Target Population': 'pop', 'Earliest Construction Date': 'econ', 'Latest Construction Date': 'lcon', 'Rent to FMR Ratio': 'rtfmr', 'Known Total Units': 'totun', 'Renewal Type Name': 'rtype', 'Inactive Status Description': 'inact', 'Construction Type': 'ctype', 'Renewal Name': 'rname', 'Renewal ID': 'rid', 'SOA Code': 'soa', 'Latitude': 'latitude', 'Longitude': 'longitude'}, inplace=True)

In [192]:
def str_to_float(x):
    try:
        x = float(x)
    except:
        x = np.nan
    return x

In [193]:
nhpd["lat"] = nhpd["Latitude"].apply(str_to_float)
nhpd["lon"] = nhpd["Longitude"].apply(str_to_float)
# Drop rows with NaN
nhpd = nhpd.dropna(subset=["lat", "lon"])
nhpd = gpd.GeoDataFrame(
    nhpd, geometry=gpd.points_from_xy(nhpd.lon, nhpd.lat)
)

In [194]:
# Export carto_nhpd to CARTO
os.chdir(CARTO)
carto_nhpd.to_csv("carto_nhpd.csv", index=False)

In [195]:
# Export nhpd to REPORT as a geojson
os.chdir(REPORT)
nhpd.to_file("nhpd.geojson", driver="GeoJSON")

/Users/anushreechaudhuri/opt/miniconda3/envs/gis/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [2]:
import pandas as pd
dac_select = pd.DataFrame(
    {
        "GEOID": ["012345678910"],
        "city": ["San Diego"],
        "county_name": ["San Diego"],
        "population": [4444.22],
        "DAC_status": ["Disadvantaged"],
        "QCT_status": ["Not Eligible"],
        "lead_paint_pct_natl_pctile": [78.81],
        "avg_energy_burden_natl_pctile": [11.11],
        "avg_housing_burden_natl_pctile": [50],
        "avg_transport_burden_natl_pctile": [55.53],
        "incomplete_plumbing_pct_natl_pctile": [0.0],
        "lowincome_ami_pct_natl_pctile": [23.1],
        "nongrid_heat_pct_natl_pctile": [100.00],
        "nonwhite_pct_natl_pctile": [35.4],
        "tract_national_percentile": [0.0],
        "tract_state_percentile": [100.00],
    }
)

In [6]:
dac

GEOID                                   012345678910
city                                       San Diego
county_name                                San Diego
population                                   4444.22
DAC_status                             Disadvantaged
QCT_status                              Not Eligible
lead_paint_pct_natl_pctile                     78.81
avg_energy_burden_natl_pctile                  11.11
avg_housing_burden_natl_pctile                    50
avg_transport_burden_natl_pctile               55.53
incomplete_plumbing_pct_natl_pctile              0.0
lowincome_ami_pct_natl_pctile                   23.1
nongrid_heat_pct_natl_pctile                   100.0
nonwhite_pct_natl_pctile                        35.4
tract_national_percentile                        0.0
tract_state_percentile                         100.0
Name: 0, dtype: object

In [4]:
for _, dac in dac_select.iterrows():
    print(f"{dac['GEOID']}")

NameError: name 'dac_select' is not defined

In [5]:
os.chdir(SQL_OUTPUT)
dac_pct = pd.read_csv("dac_pct.csv")
dac_pct.head()

,Unnamed: 0,GEOID,linguistic_isolation_pct_natl_pctile,over64_pct_natl_pctile,lead_paint_pct_natl_pctile,ej_index_diesel_natl_pctile,ej_index_cancer_natl_pctile,ej_index_traffic_natl_pctile,ej_index_water_natl_pctile,ej_index_npl_natl_pctile,...,nonwhite_pct_natl_pctile,nongrid_heat_pct_natl_pctile,lessHS_pct_natl_pctile,lowincome_fpl_pct_natl_pctile,population_natl_pctile,lowincome_ami_pct_natl_pctile,fema_loss_of_life_natl_pctile,tract_input_percentile_sum,tract_national_percentile,tract_state_percentile
0,0,35029000500,0.828050,0.855324,0.419331,0.041393,0.340161,0.299896,0.457892,0.085425,...,0.133844,0.876504,0.969078,0.935843,0.504154,0.783913,0.611382,20.737707,0.911149,0.963855
1,1,48311950100,0.718581,0.782684,0.391823,0.013362,0.032498,0.039936,0.625113,0.013706,...,0.028006,0.440134,0.540264,0.318007,0.014715,0.319962,0.354677,15.559203,0.367477,0.194339
2,2,48131950500,0.910134,0.943279,0.511218,0.041489,0.379052,0.000000,0.355627,0.088903,...,0.088331,0.737431,0.905468,0.756006,0.334433,0.720375,0.533817,20.851653,0.917377,0.802052
3,3,48247950200,0.912210,0.544349,0.359077,0.032897,0.130846,0.000000,0.000000,0.028168,...,0.023900,0.089043,0.886620,0.901718,0.157648,0.335337,0.302363,18.196890,0.688878,0.488982
4,4,48247950400,0.954785,0.893652,0.608507,0.018380,0.131465,0.000000,0.000000,0.028058,...,0.036274,0.403887,0.954076,0.894709,0.233468,0.611084,0.630566,20.891279,0.919307,0.803951


In [ ]:
import matplotlib.pyplot as plt
import pdfplot

# For every column in dac_pct except for "GEOID" and "Unnamed: 0", create a histogram of the column with 10 bins
# Add Title with column name
# Save as png in a concatenated pdf file
for col in dac_pct.columns[2:]:
    fig, ax = pdfplot.make_fig()
    ax.hist(dac_pct[col], bins=10)
    ax.set_title(col)
pdfplot.save_figs(folder="figures")




In [14]:
for col in dac_pct.columns:
    # Print number of zeros in each column
    print(f"{col}: {(dac_pct[col]==0).sum()}")

Unnamed: 0: 1
GEOID: 0
linguistic_isolation_pct_natl_pctile: 18709
over64_pct_natl_pctile: 695
lead_paint_pct_natl_pctile: 2456
ej_index_diesel_natl_pctile: 323
ej_index_cancer_natl_pctile: 323
ej_index_traffic_natl_pctile: 2397
ej_index_water_natl_pctile: 20186
ej_index_npl_natl_pctile: 300
ej_index_remediation_natl_pctile: 300
ej_index_tsdf_natl_pctile: 300
ej_index_pm25_natl_pctile: 559
over_30min_commute_pct_natl_pctile: 616
no_car_pct_natl_pctile: 12523
avg_energy_burden_natl_pctile: 697
fossil_emp_rank_natl_pctile: 4766
coal_emp_rank_natl_pctile: 4715
grid_outages_county_natl_pctile: 40463
grid_outage_duration_natl_pctile: 41138
food_desert_pct_natl_pctile: 23277
job_access_natl_pctile: 346
avg_housing_burden_natl_pctile: 1056
renters_pct_natl_pctile: 967
avg_transport_burden_natl_pctile: 883
no_internet_pct_natl_pctile: 1232
green_space_natl_pctile: 1
unhoused_pct_natl_pctile: 1652
uninsured_pct_natl_pctile: 1067
unemployed_pct_natl_pctile: 1325
disability_pct_natl_pctile: 900
i